In [5]:
!pip install -U pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26787 sha256=f88eb884168141586cf5ee125d7cf69d1628a98df01bf82a941916686e630cee
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


https://odre.opendatasoft.com/explore/dataset/consommation-quotidienne-brute-regionale/table/?disjunctive.code_insee_region&disjunctive.region&sort=-consommation_brute_gaz_grtgaz

https://api.meteo-concept.com/documentation

https://www.meteomatics.com/en/api/getting-started/?msclkid=df3cf88b68ba1259531f9bd0826350c4&utm_source=bing&utm_medium=cpc&utm_campaign=API%20M%C3%A9t%C3%A9o%20(F,CH)&utm_term=m%C3%A9t%C3%A9o%20api&utm_content=Api%20Meteo

https://github.com/Master2-IAFA/65307369

In [6]:
import json
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
from pandasql import sqldf
from termcolor import colored

In [7]:
from contextlib import closing
from urllib.request import urlopen
import json
import dateutil.parser

### ***energie_odre_nb kh***

In [ ]:
def get_data_query_energie():
    url = 'https://odre.opendatasoft.com/api/records/1.0/search/?dataset=consommation-quotidienne-brute-regionale&q=&sort=-consommation_brute_gaz_grtgaz&facet=date_heure&facet=code_insee_region&facet=region'
    lista = ['code_insee_region','date_heure', 'date', 'heure',  'region', 'consommation_brute_electricite_rte', 'consommation_brute_totale', 'consommation_brute_gaz_totale']
    response = rq.get(url)
    Reponse = response.content
    parse_json = json.loads(Reponse)
    n = parse_json['records'].__len__()
    list_data = [x for x in parse_json['records']]
    list_dic, df = {},{}

    for i in range(n):
        list_dic[i] = {key: list_data[i]["fields"][key] for key in lista}

    data = pd.DataFrame.from_dict(list_dic[0], orient='index').T

    for i in range(1,n):
        df[i] = pd.DataFrame.from_dict(list_dic[i], orient='index').T
        data = pd.concat([data,df[i]], axis=0)

#     print(data)
#     print(f'select columns from {lista}')
    cols = str(input('\n SELECT (columns):'))
    where_condition = str(input("\n FROM : (Data) \n\n WHERE (column = 'condition') : "))
    if cols and where_condition:
        query = f""" 
            SELECT {cols} 
            FROM data
            WHERE {where_condition}
            """
    elif cols:
        query = f""" 
            SELECT {cols} 
            FROM data
            """
    else:
        print('you did not select any column to query !!')
#     TO USE : TRY .....
    return sqldf(query)

In [ ]:
print(colored("""
Examples of queries to the Energy API:
Q1:
    select : *
Q2:
    select : *
    where  : region='Normandie'
Q3:
    select : region,date,consommation_brute_electricite_rte
    where  : region='Nouvelle-Aquitaine' and date='2022-03-27'
\n""", 'green'))

print(colored("""
Possibles columns: [code_insee_region, date_heure, date, heure, region, consommation_brute_electricite_rte,
                    consommation_brute_totale,consommation_brute_gaz_totale]"""
    , 'blue'))
get_data_query_energie()


Examples of queries to the Energy API:
Q1:
    select : *
Q2:
    select : *
    where  : region='Normandie'
Q3:
    select : region,date,consommation_brute_electricite_rte
    where  : region='Nouvelle-Aquitaine' and date='2022-03-27'



Possibles columns: [code_insee_region, date_heure, date, heure, region, consommation_brute_electricite_rte,
                    consommation_brute_totale,consommation_brute_gaz_totale]


KeyboardInterrupt: ignored

### ***API meteo***
idea : 


1.   faire 3 fonction chacune pour un API
2.   faire un lien entre les 3 fonction; de facon a verifier au niveau des requete la database / colomns interroges



In [65]:
    # prévision météo à un endroit grâce au code insee
url1 = 'https://api.meteo-concept.com/api/forecast/daily?token=756f550d7a0f31da58721d31e518f252068b8c1e9a17c1d366d1e8e8edc405b2&insee=35238' 
    # prévision météo dans qlq (3) jours
url2 = 'https://api.meteo-concept.com/api/forecast/daily/3/period/2?token=756f550d7a0f31da58721d31e518f252068b8c1e9a17c1d366d1e8e8edc405b2&amp;insee=31555'
# info sur ephemeride
url3 = 'https://api.meteo-concept.com/api/ephemeride/1?token=756f550d7a0f31da58721d31e518f252068b8c1e9a17c1d366d1e8e8edc405b2&insee=35238'
    # Localisation des villes
url4 = 'https://api.meteo-concept.com/api/location/city?token=756f550d7a0f31da58721d31e518f252068b8c1e9a17c1d366d1e8e8edc405b2&insee=31555'

WEATHER = {
        0: "Soleil",
        1: "Peu nuageux",
        2: "Ciel voilé",
        3: "Nuageux",
        4: "Très nuageux",
        5: "Couvert",
        6: "Brouillard",
        7: "Brouillard givrant",
        10: "Pluie faible",
        11: "Pluie modérée",
        12: "Pluie forte",
        13: "Pluie faible verglaçante",
        14: "Pluie modérée verglaçante",
        15: "Pluie forte verglaçante",
        16: "Bruine",
        20: "Neige faible",
        21: "Neige modérée",
        22: "Neige forte",
        30: "Pluie et neige mêlées faibles",
        31: "Pluie et neige mêlées modérées",
        32: "Pluie et neige mêlées fortes",
        40: "Averses de pluie locales et faibles",
        41: "Averses de pluie locales",
        42: "Averses locales et fortes",
        43: "Averses de pluie faibles",
        44: "Averses de pluie",
        45: "Averses de pluie fortes",
        46: "Averses de pluie faibles et fréquentes",
        47: "Averses de pluie fréquentes",
        48: "Averses de pluie fortes et fréquentes",
        60: "Averses de neige localisées et faibles",
        61: "Averses de neige localisées",
        62: "Averses de neige localisées et fortes",
        63: "Averses de neige faibles",
        64: "Averses de neige",
        65: "Averses de neige fortes",
        66: "Averses de neige faibles et fréquentes",
        67: "Averses de neige fréquentes",
        68: "Averses de neige fortes et fréquentes",
        70: "Averses de pluie et neige mêlées localisées et faibles",
        71: "Averses de pluie et neige mêlées localisées",
        72: "Averses de pluie et neige mêlées localisées et fortes",
        73: "Averses de pluie et neige mêlées faibles",
        74: "Averses de pluie et neige mêlées",
        75: "Averses de pluie et neige mêlées fortes",
        76: "Averses de pluie et neige mêlées faibles et nombreuses",
        77: "Averses de pluie et neige mêlées fréquentes",
        78: "Averses de pluie et neige mêlées fortes et fréquentes",
        100: "Orages faibles et locaux",
        101: "Orages locaux",
        102: "Orages fort et locaux",
        103: "Orages faibles",
        104: "Orages",
        105: "Orages forts",
        106: "Orages faibles et fréquents",
        107: "Orages fréquents",
        108: "Orages forts et fréquents",
        120: "Orages faibles et locaux de neige ou grésil",
        121: "Orages locaux de neige ou grésil",
        122: "Orages locaux de neige ou grésil",
        123: "Orages faibles de neige ou grésil",
        124: "Orages de neige ou grésil",
        125: "Orages de neige ou grésil",
        126: "Orages faibles et fréquents de neige ou grésil",
        127: "Orages fréquents de neige ou grésil",
        128: "Orages fréquents de neige ou grésil",
        130: "Orages faibles et locaux de pluie et neige mêlées ou grésil",
        131: "Orages locaux de pluie et neige mêlées ou grésil",
        132: "Orages fort et locaux de pluie et neige mêlées ou grésil",
        133: "Orages faibles de pluie et neige mêlées ou grésil",
        134: "Orages de pluie et neige mêlées ou grésil",
        135: "Orages forts de pluie et neige mêlées ou grésil",
        136: "Orages faibles et fréquents de pluie et neige mêlées ou grésil",
        137: "Orages fréquents de pluie et neige mêlées ou grésil",
        138: "Orages forts et fréquents de pluie et neige mêlées ou grésil",
        140: "Pluies orageuses",
        141: "Pluie et neige mêlées à caractère orageux",
        142: "Neige à caractère orageux",
        210: "Pluie faible intermittente",
        211: "Pluie modérée intermittente",
        212: "Pluie forte intermittente",
        220: "Neige faible intermittente",
        221: "Neige modérée intermittente",
        222: "Neige forte intermittente",
        230: "Pluie et neige mêlées",
        231: "Pluie et neige mêlées",
        232: "Pluie et neige mêlées",
        235: "Averses de grêle",
    }

list1 = ["insee","cp", "latitude", "longitude","day", "datetime", "wind10m", "gust10m", "dirwind10m", "rr10", "rr1", "probarain",
            "weather", "tmin", "tmax", "sun_hours", "etp", "probafrost", "probafog", "probawind70", "probawind100", "gustx"]
list2 = []
list3 = []
list4 = ["insee", "cp", "latitude", "longitude", "altitude", "name"]

In [51]:
def df_query(columns, data):
  cols = str(input('\n SELECT (columns):'))
  where_condition = str(input("\n FROM : (data) \n\n WHERE (column = 'condition') : "))
  if cols and where_condition:
        query = f""" 
            SELECT {cols} 
            FROM data
            WHERE {where_condition}
            """
  elif cols:
        query = f""" 
            SELECT {cols} 
            FROM data
            """
  else:
        print('you did not select any column to query !!')
#     TO USE : TRY .....
  return sqldf(query)

In [ ]:
# Localisation des villes 
def localiser_ville(code):
  # code = 31555; toulouse
  url4 = 'https://api.meteo-concept.com/api/location/city?token=756f550d7a0f31da58721d31e518f252068b8c1e9a17c1d366d1e8e8edc405b2&='+str(code)
  print(url4)
  with closing(urlopen(url4)) as f:
	  city = json.loads(f.read())['city']
	  print(u'\nLa ville de {} ({}) a pour coordonnées {},{}.'.format(city['name'], city['cp'], city['latitude'], city['longitude']))
  df_4 = pd.DataFrame.from_dict(pd.json_normalize(city), orient = 'columns')
  return df_4

# print(df_4)
code = 31553
df = localiser_ville(code)
df_query(df.columns, df)

In [73]:
# prévision météo à un endroit grâce au code insee
def prevision_ville(code):
  # code = 35238
  url1 = 'https://api.meteo-concept.com/api/forecast/daily?token=756f550d7a0f31da58721d31e518f252068b8c1e9a17c1d366d1e8e8edc405b2&insee='+str(code)
  with closing(urlopen(url1)) as f:
	  forecast = json.loads(f.read())['forecast']
	  print(len(forecast))
	  c = 0
	  for x in forecast:
		  if c != x['insee']:
			  print("Le temps  à " , x['insee'], " est :", WEATHER[x['weather']])
		  c = x['insee']

  df_1 = pd.DataFrame.from_dict(pd.json_normalize(forecast), orient = 'columns')
  return df_1
  
code = 31553
df = prevision_ville(code)
df_query(df.columns, df)

14
Le temps  à  31209  est : Averses de pluie fréquentes

 SELECT (columns):*

 FROM : (data) 

 WHERE (column = 'condition') : 


,insee,cp,latitude,longitude,day,datetime,wind10m,gust10m,dirwind10m,rr10,...,weather,tmin,tmax,sun_hours,etp,probafrost,probafog,probawind70,probawind100,gustx
0,31209,31480,43.6893,1.0917,0,2022-11-25T01:00:00+0100,5,40,277,6.1,...,47,-25,12,2,0,100,50,0,0,47
1,31209,31480,43.6893,1.0917,1,2022-11-26T01:00:00+0100,10,29,80,0.0,...,3,6,11,3,0,10,70,0,0,29
2,31209,31480,43.6893,1.0917,2,2022-11-27T01:00:00+0100,15,35,129,0.0,...,2,6,12,5,1,10,0,0,0,35
3,31209,31480,43.6893,1.0917,3,2022-11-28T01:00:00+0100,20,33,289,17.7,...,106,8,11,0,0,10,0,10,0,44
4,31209,31480,43.6893,1.0917,4,2022-11-29T01:00:00+0100,10,27,271,1.7,...,41,5,9,4,0,10,0,0,0,42
5,31209,31480,43.6893,1.0917,5,2022-11-30T01:00:00+0100,10,21,135,0.2,...,3,4,9,3,0,30,10,0,0,21
6,31209,31480,43.6893,1.0917,6,2022-12-01T01:00:00+0100,10,18,178,0.8,...,40,5,10,4,0,10,20,0,0,18
7,31209,31480,43.6893,1.0917,7,2022-12-02T01:00:00+0100,10,15,129,0.2,...,3,5,10,5,0,10,20,0,0,15
8,31209,31480,43.6893,1.0917,8,2022-12-03T01:00:00+0100,10,21,128,0.2,...,3,5,10,5,1,10,20,0,0,21
9,31209,31480,43.6893,1.0917,9,2022-12-04T01:00:00+0100,10,21,127,1.6,...,40,6,10,5,1,10,20,0,0,31


In [75]:
# prévision météo dans qlq jours 
def prev_qlq_jours(jours, code):
  url2 = 'https://api.meteo-concept.com/api/forecast/daily/'+str(jours)+'/period/2?token=756f550d7a0f31da58721d31e518f252068b8c1e9a17c1d366d1e8e8edc405b2&amp;insee='+str(code)
  with closing(urlopen(url2)) as f:
    forecast = json.loads(f.read())['forecast']
    print(u"\nLe temps prévu pour l'après-midi dans "+str(jours)+" jours est : \"{}\"".format(WEATHER[forecast['weather']]))

  df_2 = pd.DataFrame.from_dict(pd.json_normalize(forecast), orient = 'columns')
  return df_2

code = 31553
df = prev_qlq_jours(2, code)
df_query(df.columns, df)


Le temps prévu pour l'après-midi dans 2 jours est : "Averses de pluie faibles"

 SELECT (columns):*

 FROM : (data) 

 WHERE (column = 'condition') : 


,insee,cp,latitude,longitude,day,period,datetime,temp2m,wind10m,gust10m,dirwind10m,rr10,rr1,probarain,weather,probafrost,probafog,probawind70,probawind100,gustx
0,35238,35000,48.112,-1.6819,2,2,2022-11-27T13:00:00+0100,13,25,37,195,2,3,70,43,0,0,0,0,47


In [76]:
# infos sur éphéméride

def info_ephem(code):
  # code = 35238
  url3 = 'https://api.meteo-concept.com/api/ephemeride/1?token=756f550d7a0f31da58721d31e518f252068b8c1e9a17c1d366d1e8e8edc405b2&insee='+str(code)

  with closing(urlopen(url3)) as f:
    cityEph = json.loads(f.read())
    print(u'Demain, à {}, le soleil se lèvera à {} et se couchera à {}.'.format(cityEph['city']['name'], cityEph['ephemeride']['sunrise'], cityEph['ephemeride']['sunset']))
    print(u'On comptera {} minutes de soleil de {} qu\'aujourd\'hui.'.format(abs(cityEph['ephemeride']['diff_duration_day']), 'moins' if cityEph['ephemeride']['diff_duration_day'] <= 0 else 'plus'))

  df_eph = pd.DataFrame.from_dict(pd.json_normalize(cityEph), orient = 'columns')
  return df_eph

code = 31553
df = info_ephem(code)
df_query(df.columns, df)

Demain, à Thil, le soleil se lèvera à 08:01 et se couchera à 17:23.
On comptera 2 minutes de soleil de moins qu'aujourd'hui.

 SELECT (columns):*

 FROM : (data) 

 WHERE (column = 'condition') : 


,city.insee,city.cp,city.name,city.latitude,city.longitude,city.altitude,ephemeride.latitude,ephemeride.longitude,ephemeride.insee,ephemeride.day,ephemeride.datetime,ephemeride.sunrise,ephemeride.sunset,ephemeride.duration_day,ephemeride.diff_duration_day,ephemeride.moon_age,ephemeride.moon_phase
0,31553,31530,Thil,43.7102,1.153,212,43.7102,1.153,31553,1,2022-11-26T00:00:00+0100,08:01,17:23,09:22,-2,2.3,Premier croissant


In [79]:
# Prévisions météo par jour 
def prev_weekend(day, code):
  # 31555
  url = 'https://api.meteo-concept.com/api/forecast/daily?token=756f550d7a0f31da58721d31e518f252068b8c1e9a17c1d366d1e8e8edc405b2&insee&insee='+str(code)
  with closing(urlopen(url)) as f:
    decoded = json.loads(f.read())
    (city,forecast) = (decoded[k] for k in ('city','forecast'))

    saturday = None
    for i,f in enumerate(forecast):
        day = dateutil.parser.parse(f['datetime']).weekday() # Lundi : 0, Mardi : 1, etc.
        if day == 5:
            saturday = i
            break

    print(u"Le week-end prochain est dans {} jours ! Les températures mini/maxi à {} seront :".format(saturday, city['name']))
    print(u"\tSamedi   : {}°C/{}°C".format(forecast[saturday]['tmin'],forecast[saturday]['tmax']))
    print(u"\tDimanche : {}°C/{}°C".format(forecast[saturday+1]['tmin'],forecast[saturday+1]['tmax']))

  # day = saturday
  df_j = pd.DataFrame.from_dict(pd.json_normalize(forecast[day]), orient = 'columns')
  return df_j

code = 31553
day = saturday

df = prev_weekend(day, code)
df_query(df.columns, df)

Le week-end prochain est dans 1 jours ! Les températures mini/maxi à Thil seront :
	Samedi   : 6°C/11°C
	Dimanche : 6°C/12°C

 SELECT (columns):*

 FROM : (data) 

 WHERE (column = 'condition') : 


,insee,cp,latitude,longitude,day,datetime,wind10m,gust10m,dirwind10m,rr10,...,weather,tmin,tmax,sun_hours,etp,probafrost,probafog,probawind70,probawind100,gustx
0,31209,31480,43.6893,1.0917,5,2022-11-30T01:00:00+0100,10,21,135,0.2,...,3,4,9,3,0,30,10,0,0,21


In [80]:
#Prévisions météo par ville et  par heure 

def prev_ville_heure(code,datetime ):
  # 35238

  url = 'https://api.meteo-concept.com/api/forecast/nextHours?token=756f550d7a0f31da58721d31e518f252068b8c1e9a17c1d366d1e8e8edc405b2&insee='+str(code)
  with closing(urlopen(url)) as f:
    forecast = json.loads(f.read())['forecast']
    print(u"Humidité relative de l'air (%)\n   ∧")
    for i in range(100, 0, -10):
        print(u"{: >3}│ ".format(i), end='')
        for f in forecast:
            if f['rh2m'] >= i:
                print(u"  █    ", end='')
            else:
                print(u"       ", end='')
        print("")
    print("   └─", end='')
    for f in forecast:
        print("───────", end='')
    print(">\n     ", end='')
    for f in forecast:
        print(dateutil.parser.parse(f['datetime']).strftime('%H:%M  '), end='')
    print("")
    
  #datetime = 2022-11-25
  df_vh = pd.DataFrame.from_dict(pd.json_normalize(forecast), orient = 'columns')
  return df_vh

datetime = 2022-11-25
code = 35238
df = prev_ville_heure(code, datetime )
df_query(df.columns, df)

Humidité relative de l'air (%)
   ∧
100│                             
 90│                             
 80│          █      █      █    
 70│          █      █      █    
 60│   █      █      █      █    
 50│   █      █      █      █    
 40│   █      █      █      █    
 30│   █      █      █      █    
 20│   █      █      █      █    
 10│   █      █      █      █    
   └─────────────────────────────>
     14:00  17:00  20:00  23:00  

 SELECT (columns):*

 FROM : (data) 

 WHERE (column = 'condition') : 


,insee,cp,latitude,longitude,datetime,temp2m,rh2m,wind10m,gust10m,dirwind10m,...,probarain,weather,probafrost,probafog,probawind70,probawind100,tsoil1,tsoil2,gustx,iso0
0,35238,35000,48.112,-1.6819,2022-11-25T14:00:00+0100,13,68,14,36,266,...,20,3,0,0,0,0,11,11,36,-99
1,35238,35000,48.112,-1.6819,2022-11-25T17:00:00+0100,12,80,7,30,247,...,20,3,0,0,0,0,11,11,30,-99
2,35238,35000,48.112,-1.6819,2022-11-25T20:00:00+0100,11,88,7,25,235,...,10,3,0,0,0,0,9,11,25,-99
3,35238,35000,48.112,-1.6819,2022-11-25T23:00:00+0100,11,89,9,25,234,...,10,3,0,10,0,0,8,11,25,-99


In [111]:
# bilan ville
# not complete yet
def bilan_ville(ville):
  # Rennes
  with closing(urlopen('https://api.meteo-concept.com/api/location/cities?token=756f550d7a0f31da58721d31e518f252068b8c1e9a17c1d366d1e8e8edc405b2&search='+ville)) as f:
     cities = json.loads(f.read())['cities']

  df_v = pd.DataFrame.from_dict(pd.json_normalize(cities), orient = 'columns')
  print(len(df_v))
  print(df_v.columns)
  #df_v  
  



  return df_v

In [112]:
ville = 'Toulouse'
bilan_ville(ville)

2
Index(['insee', 'cp', 'latitude', 'longitude', 'altitude', 'name'], dtype='object')


,insee,cp,latitude,longitude,altitude,name
0,31555,31000,43.5964,1.4317,148,Toulouse
1,39533,39230,46.8222,5.5831,258,Toulouse-le-Château
